### LGB用のデータ作成

### インポート

In [211]:
# ライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mpl_finance
%matplotlib inline
plt.rcParams['font.family'] = 'IPAPGothic'
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
import oandapyV20.endpoints.instruments as instruments
import datetime
from statistics import mean
from statistics import median
import calendar
import datetime
from math import floor
import schedule
import time
import threading
import json
import random
import sklearn
import lightgbm as lgb

In [2]:
# OANDA API v20の口座IDとAPIトークン
accountID = "101-009-12789646-001"
access_token = "1ab53171ce4413f408de6da28ea23089-6426c3d3cc947dba212b3766d1be45c6"
# OANDAのデモ口座へのAPI接続
api = API(access_token=access_token, environment="practice")

### 怒濤の関数定義

In [3]:
# APIから取得したレートをPandasのDataFrameへ
def to_dataframe(r):
    data = []
    for raw in r.response['candles']:
        data.append([raw['time'], raw['volume'], raw['mid']['o'], raw['mid']['h'], raw['mid']['l'], raw['mid']['c']])

    # リストからPandas DataFrameへ変換
    df = pd.DataFrame(data)
    df.columns = ['Time', 'Volume', 'Open', 'High', 'Low', 'Close']
    df['Time'] = pd.to_datetime(df['Time'])
    for col in ['Open', 'High', 'Low', 'Close']:
        df[col] = df[col].apply(float)
    return df

In [4]:
def get_data(count, gran, year, month, day, hour, minute, second):
    # OANDA API v20の口座IDとAPIトークン
    accountID = "101-009-12789646-001"
    access_token = "1ab53171ce4413f408de6da28ea23089-6426c3d3cc947dba212b3766d1be45c6"
    # OANDAのデモ口座へのAPI接続
    api = API(access_token=access_token, environment="practice")
    fmt = '%Y-%m-%dT%H:%M:00.000000Z'
    _from = datetime.datetime(year=year, month=month, day=day,
                              hour=hour, minute=minute, second=second).strftime(fmt)
    params = {
        "count": count,
        "granularity": gran,
        'from': _from,
#         'dailyAlignment': 0
    }
    r = instruments.InstrumentsCandles(instrument="USD_JPY", params=params)
    api.request(r)
    df = to_dataframe(r)
    return df

In [9]:
def get_data_super(start, end, gran):
    # OANDA API v20の口座IDとAPIトークン
    accountID = "101-009-12789646-001"
    access_token = "1ab53171ce4413f408de6da28ea23089-6426c3d3cc947dba212b3766d1be45c6"
    # OANDAのデモ口座へのAPI接続
    api = API(access_token=access_token, environment="practice")
    # xxxx-xx-xx～xxxx-xx-xx の期間のデータを取得してデータフレームを返す
    s_year, s_month, s_day, s_hour, s_minute, s_second = map(int, start.split('-'))
    e_year, e_month, e_day, e_hour, e_minute, e_second = map(int, end.split('-'))
    fmt = '%Y-%m-%dT%H:%M:00.000000Z'
    from_ = datetime.datetime(year=s_year, month=s_month, day=s_day,
                              hour=s_hour, minute=s_minute, second=s_second).strftime(fmt)
    to_ = datetime.datetime(year=e_year, month=e_month, day=e_day,
                            hour=e_hour, minute=e_minute, second=e_second).strftime(fmt)
    df = pd.DataFrame()
    # 5000個制限に引っかからなければこっちの処理
    try:
        params = {
            "granularity": gran,
            'from': from_,
            'to': to_
        }
        r = instruments.InstrumentsCandles(instrument="USD_JPY", params=params)
        api.request(r)
        df = to_dataframe(r)
    # 引っかかればこっちの処理
    except:
        i = True
        while i:
            df_tmp = get_data(count=5000, gran=gran, year=s_year, month=s_month, day=s_day,
                             hour=s_hour, minute=s_minute, second=s_second)
            df = pd.concat([df, df_tmp], ignore_index=True)
            index = df_tmp.tail(1).index[0]
            s_year = df_tmp.iat[index, 0].year
            s_month = df_tmp.iat[index, 0].month
            s_day = df_tmp.iat[index, 0].day
            s_hour = df_tmp.iat[index, 0].hour
            s_minute = (df_tmp.iat[index, 0].minute + 1) % 60
            s_second = df_tmp.iat[index, 0].second
#             print('{}-{}-{}-{}-{}-{}'.format(s_year, s_month, s_day, s_hour, s_minute, s_second))
            # 5000個ずつ取得してエンド超えたら止める
            if s_year == e_year:
                i = False
        # 止めたらはみ出した分をここで切り落とす
        for i in range(len(df.index) - 5000, len(df.index)):
            if df.iat[i, 0].year == e_year and df.iat[i, 0].month == e_month and df.iat[i, 0].day == e_day:
                df = df[0:i]
                break
    return df

In [6]:
def get_evaluation(pos_df):
    # 勝率
    win_c = pos_df[pos_df['pips'] >=0].shape[0]
    WP = round(win_c / len(pos_df.index) * 100, 2)
    print('勝率: {}%'.format(WP))
    # ペイオフレシオ
    win_p = mean(pos_df[pos_df['pips'] >=0]['pips'])
    los_p = mean(pos_df[pos_df['pips'] < 0]['pips']) * (-1)
    RR = round(win_p / los_p, 4)
    print('ペイオフレシオ: {}'.format(RR))
    # 期待値
    win_pro = mean(pos_df[pos_df['pips'] >=0]['profit'])
    los_pro = mean(pos_df[pos_df['pips'] < 0]['profit']) * (-1)
    E = round(P * win_pro - pos_df[pos_df['pips'] < 0].shape[0] / len(pos_df.index) * los_pro, 2)
    print('期待値: {}円'.format(E))
    DD = round(pos_df.sort_values(by='profit', ascending=True).iat[0, 5], 2)
    print('最大ドローダウン: -{}円'.format(DD))
    return WP, RR, E, DD

In [7]:
def set_profit(tmp_i, i, pos, entry_rate, exit_rate, assets, df, pos_df, pos_count, lot):
    fee = 0.008
    pos_count+=1
    if pos == 'entry_long':
        pips = (exit_rate - entry_rate - fee) * 100
        l_s = 'long'
    else:
        pips = (entry_rate - exit_rate - fee) * 100
        l_s = 'short'
    profit = lot * 25 * pips / 100
    assets = round(assets + profit, 4)
    tmp_se = pd.Series( [lot, 'long', df.iat[tmp_i,0], entry_rate, df.iat[i+1,0],
                        exit_rate, pips, profit, assets ], index=pos_df.columns )
    pos_df = pos_df.append( tmp_se, ignore_index=True )
    pos = 'None'
    return assets, pos_df, pos_count, pos

### 直近10個のデータから10分後の終値を予測する

In [187]:
# 訓練データ作る関数
def get_dataset(df):
    LGB_df = pd.DataFrame()
    
    # 実行
    for i in range(10, len(df.index)-10):
        LGB_df_tmp = pd.DataFrame(columns=['10Close'])
        y = df.iat[i+10, 5] # こやつが正解ラベル
        tmp_se = pd.Series( [y], index=LGB_df_tmp.columns )
        LGB_df_tmp = LGB_df_tmp.append( tmp_se, ignore_index=True )
        # 直近10個を横にずらーっと追加
        for j in range(i-10, i):
            tmp_df = df[j:j+1]
            tmp_df.reset_index(drop=True, inplace=True)
            LGB_df_tmp = pd.concat([LGB_df_tmp, tmp_df], axis=1)
        # 縦に結合
        LGB_df = pd.concat([LGB_df, LGB_df_tmp], axis=0)

    return LGB_df

In [10]:
df = get_data_super(start='2018-07-01-00-00-00', end='2019-01-01-00-00-00', gran='M1')

In [122]:
df.head(21)

,Time,Volume,Open,High,Low,Close
0,2018-07-01 21:00:00+00:00,7,110.675,110.701,110.624,110.701
1,2018-07-01 21:01:00+00:00,11,110.705,110.705,110.636,110.662
2,2018-07-01 21:02:00+00:00,11,110.662,110.668,110.642,110.655
3,2018-07-01 21:03:00+00:00,2,110.650,110.650,110.646,110.646
4,2018-07-01 21:04:00+00:00,7,110.641,110.641,110.622,110.628
5,2018-07-01 21:05:00+00:00,60,110.633,110.640,110.621,110.639
6,2018-07-01 21:06:00+00:00,4,110.635,110.635,110.622,110.626
7,2018-07-01 21:07:00+00:00,1,110.630,110.630,110.630,110.630
8,2018-07-01 21:08:00+00:00,4,110.636,110.648,110.636,110.648
9,2018-07-01 21:09:00+00:00,39,110.644,110.686,110.629,110.658


In [135]:
df[0:1]

,Time,Volume,Open,High,Low,Close
0,2018-07-01 21:00:00+00:00,7,110.675,110.701,110.624,110.701


In [168]:
LGB_df = pd.DataFrame()

In [174]:
LGB_df_tmp = pd.DataFrame(columns=['10Close'])
i = 13
y = df.iat[i+10, 5] # こやつが正解ラベル
tmp_se = pd.Series( [y], index=LGB_df_tmp.columns )
LGB_df_tmp = LGB_df_tmp.append( tmp_se, ignore_index=True )
# 直近10個を横にずらーっと追加
for j in range(i-10, i):
    tmp_df = df[j:j+1]
    tmp_df.reset_index(drop=True, inplace=True)
    LGB_df_tmp = pd.concat([LGB_df_tmp, tmp_df], axis=1)
# 縦に結合
LGB_df = pd.concat([LGB_df, LGB_df_tmp], axis=0)

In [138]:
i = 11
y = df.iat[i+10, 5]
tmp_se = pd.Series( [y] )
pd.concat([LGB_df, tmp_df])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


ValueError: Plan shapes are not aligned

In [210]:
LGB_df = get_dataset(df)

KeyboardInterrupt: 